# Recuperation de la page

On recupere la page via le beautifulsoup


In [1]:
import requests
import json
from bs4 import BeautifulSoup

In [2]:
from pprint import pprint

In [3]:
class Content:
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=4, ensure_ascii=False)
def getPage(url):
    req = requests.get(url)
    return BeautifulSoup(req.text, 'html.parser')

class Dictlist(dict):
    def __setitem__(self, key, value):
        try:
            self[key]
        except KeyError:
            super(Dictlist, self).__setitem__(key, [])
        self[key].append(value)

In [4]:
def scrapeBrookings2(url):
    bs = getPage(url)
    body = bs.find('table', {'class', 'table table-condensed'})
    body = body.table.find_all('td')
    
    title = body[1].text
    date_creation = body[3].text
    projets = body[5].text
    debut = body[7].text
    fin = body[9].text
    resume = body[17].text
    strategie = body[19].text
    
    i = 22
    sites = []
    while 'Site ' in body[i].text:
        sites.append(Site(body[i+2].text, body[i+4].text,body[i+6].text,body[i+8].text,body[i+10].text,body[i+12].text,body[i+14].text,body[i+16].text))
        i = i+17
    params = []
    while 'Paramètre mesuré ' in body[i].text:
        params.append(Parametre(body[i+2].text,body[i+4].text,body[i+6].text,body[i+8].text,body[i+10].text,body[i+12].text))
        i = i+13
    
    return Content(url, title, date_creation,projets,debut,fin,resume,strategie,sites,params)

In [5]:
url = "https://dataverse.ird.fr/dataset.xhtml?persistentId=doi:10.23708/M9N7AI"
req = requests.get(url)
bs = BeautifulSoup(req.text, 'html.parser')

In [6]:
body = bs.find_all('div', {'class', 'panel-group'})
#body = body.find_all('td')
len(body)
metadata = body[0].find_all('td')

In [7]:
def getContacts (data):
    values = data.split("\n")
    dict_contacts = Dictlist()
    results = list()
    
    for i in range(len(values)):
        contact = Content()
        if values[i] == None or values[i].replace(' ','') == '':
            break
        ct = values[i].split("ORCID:")
        ct_inst = ct[0].split("(",1)
        if(len(ct)>1):
            contact.orcid = ct[1]
        else:
            contact.orcid = '-'
        contact.name = ct_inst[0].replace(' ','')
        contact.organisations = ct_inst[1].replace(')','').split('-')
        results.append(contact.__dict__)
    return results

In [8]:
content = Content()

In [9]:
metadata[15].text.replace('\n','').replace('  ','').split(" ")

['-16.398672', '-16.5308055', '14.4788611', '14.5962972']

In [10]:
content.doi = metadata[0].text.replace('\n','').replace(' ','')
content.publication_date = metadata[1].text.replace('\n','')
content.title = metadata[2].text.replace('\n','')

content.publisher = getContacts (metadata[3].text)
content.personne_ressource = metadata[4].text.split('\n')[2].replace(' ','')

content.description = metadata[5].text
content.series = metadata[6].text
content.keywords = metadata[7].text

content.thematique = metadata[8].text

content.publications = metadata[9].text

content.remarques = metadata[10].text

content.deposant = metadata[11].text.replace('\n','').replace(' ','')

content.date_depot = metadata[12].text.replace('\n','').replace(' ','')

content.temporal_extent = metadata[13].text.replace('\n','').replace(' ','').replace('"','').split(';')
content.sites = metadata[14].text.replace('\n','').replace(' ','')
content.spatial_extents = metadata[15].text.replace('\n','').replace('  ','').split(" ")

## Sauvegarde du fichier Json

In [11]:
path = "/home/openopse/Documents/OPENOPSE/source code/data/test2.json"

In [12]:
with open(path, 'w',encoding='utf8') as outfile:
    json.dump(content.__dict__, outfile,indent=4, ensure_ascii=False)